In [1]:
import pandas as pd
import re
import pickle
import string

In [2]:
Dataset = 'AudioCaps' # or 'Clotho'

In [3]:
def repair_caption(caption) : 
                       
    caption = caption.lower()    
                
    caption = caption.replace(',', ' , ') 
    caption = re.sub(' +', ' ', caption)
    caption = caption.replace(' ,', ',')
    caption = re.sub(r'[.]', '', caption)

    caption = caption.strip()

    caption += '.'

    return caption

In [4]:
train_csv = None
caption_list = []

if Dataset == 'AudioCaps' :
    train_csv = pd.read_csv('train.csv')

    captions = train_csv['caption'].to_list()
    for caption in captions:

        edited_caption = repair_caption(caption)

        # if edited_caption != caption :
        #     print("before repair :", caption)
        #     print("after repair :", edited_caption)
        #     print()

        caption_list.append(edited_caption)

elif Dataset == 'Clotho' :
    train_csv = pd.read_csv('clotho_captions_development.csv')

    for i in range(5) :
        col_str = 'caption_' + str(i + 1)
    
        captions = train_csv[col_str].to_list()
        for caption in captions:

            edited_caption = repair_caption(caption)

            # if edited_caption != caption :
            #     print("before repair :", caption)
            #     print("after repair :", edited_caption)
            #     print()

            caption_list.append(caption)


    


In [5]:
caption_list[0]

'a woman talks nearby as water pours.'

In [6]:
word_dictionary = {}

for caption in caption_list :

    word_list = caption.strip().split(' ')

    for word in word_list : 
        
        if word in word_dictionary: 
            word_dictionary[word] += 1
        else : 
            word_dictionary[word] = 1

In [7]:
word_dictionary_sorted = sorted(word_dictionary.items(), key = lambda item: item[1], reverse = True)

word_Dict = []

# 단어만 담기
# 그러면 등장 빈도수가 높은 단어부터 나열되는 딕셔너리가 완성됨
for key, value in word_dictionary_sorted :
    word_Dict.append(key)

print(len(word_Dict))

7908


In [8]:
# # GPT-2는 vocab의 맨 첫 idx의 값을 "!"로 했다. 나도 그렇게 해야겠다.
# # 그리고 GPT-2는 문장의 마무리라 할 수 있는 마침표('.')의 idx를 13으로 했다. 나는 문장의 마무리를 <eos>로 해야겠다. 
# word_Dict.insert(0, '!')
# word_Dict.insert(13, '<eos>')
# word_Dict.append('<unk>')

In [9]:
file_name = None
if Dataset == 'AudioCaps' :
    file_name = 'AudioCaps_vocabulary_'
elif Dataset == 'Clotho' :
    file_name = 'Clotho_vocabulary_'

file_name += (str(len(word_Dict)) + '.pickle')

with open(file_name, 'wb') as f:
    pickle.dump(word_Dict, f, pickle.HIGHEST_PROTOCOL)

In [10]:
len(word_Dict)

7908

In [11]:
# Clotho Dataset에만 있는 word들을 추가해줌
caption_list_clotho = []
if Dataset == 'AudioCaps' :
    train_csv = pd.read_csv('clotho_captions_development.csv')

    for i in range(5) :
        col_str = 'caption_' + str(i + 1)
    
        captions = train_csv[col_str].to_list()
        for caption in captions:

            edited_caption = repair_caption(caption)
            caption_list_clotho.append(edited_caption)

    word_dictionary_clotho = {}

    for caption in caption_list_clotho :

        word_list = caption.strip().split(' ')

        for word in word_list : 
            
            if word in word_dictionary_clotho: 
                word_dictionary_clotho[word] += 1
            else : 
                word_dictionary_clotho[word] = 1

    word_dictionary_sorted_clotho = sorted(word_dictionary_clotho.items(), key = lambda item: item[1], reverse = True)

    not_in_count = 0
    in_count = 0
    for key, value in word_dictionary_sorted_clotho :

        if key not in word_Dict :
            word_Dict.append(key)
            not_in_count +=1
        else :
            in_count +=1
    
    print(str(not_in_count), "words not in audiocaps vocab")
    print(str(in_count), "words in audiocaps vocab")

2729 words not in audiocaps vocab
4293 words in audiocaps vocab


In [12]:
# GPT-2는 vocab의 맨 첫 idx의 값을 "!"로 했다. 나도 그렇게 해야겠다.
# 그리고 GPT-2는 문장의 마무리라 할 수 있는 마침표('.')의 idx를 13으로 했다. 나는 문장의 마무리를 <eos>로 해야겠다. 
word_Dict.insert(0, '!')
word_Dict.insert(13, '<eos>')
word_Dict.append('<unk>')

In [13]:
file_name = 'Clotho_vocabulary_' + (str(len(word_Dict)) + '.pickle')

with open(file_name, 'wb') as f:
    pickle.dump(word_Dict, f, pickle.HIGHEST_PROTOCOL)